In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transaction-naration-data-set/Transaction_Naration_Data_Set.csv


In [2]:
data=pd.read_csv('/kaggle/input/transaction-naration-data-set/Transaction_Naration_Data_Set.csv')
data.head()

,Unnamed: 0,AC_ID,AMOUNT,DOC_DATE,NARATION,DR_CR
0,0,3644877,342800.0,01-NOV-22,Cash Withdraw,Debit
1,1,1175256,342800.0,01-NOV-22,Cash Withdrawal From A/C No.:,Credit
2,2,2298226,1500.0,01-NOV-22,"Cash withdraw from Micro Merchant point, MM A/C",Debit
3,3,4066427,1500.0,01-NOV-22,Credit Against Merchant Cash Withdraw Service ...,Credit
4,4,1978564,2550.0,01-NOV-22,"Cash withdraw from Micro Merchant point, MM A/C",Debit


In [3]:
sub_data=data[['NARATION','DR_CR']].copy()
# sub_data.info()
sub_data.isnull().sum()

NARATION    1023
DR_CR          0
dtype: int64

In [4]:
sub_data.dropna(subset=['NARATION'],inplace=True)
sub_data.isnull().sum()
sub_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22545735 entries, 0 to 22546757
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   NARATION  object
 1   DR_CR     object
dtypes: object(2)
memory usage: 516.0+ MB


In [9]:
sub_data['NARATION'][50]

'Cash Withdrawal From A/C No.: '

In [6]:
sub_data['naration_plain']=sub_data['NARATION'].str.replace('[^A-Za-z\s]', '',regex=True)
sub_data['naration_plain'][1]

'Cash Withdrawal From AC No '

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 

stop_words=set(stopwords.words('english'))

ps = PorterStemmer()



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
sub_data['naration_plain'][1000]

'Cash Deposit to AC No '

In [8]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
%pwd


'/kaggle/working'

In [17]:
%cd /usr/share/nltk_data/corpora

/usr/share/nltk_data/corpora


In [18]:
%ls

abc/                     kimmo/                 sentence_polarity/
abc.zip                  kimmo.zip              sentence_polarity.zip
alpino/                  knbc.zip               sentiwordnet/
alpino.zip               lin_thesaurus/         sentiwordnet.zip
biocreative_ppi/         lin_thesaurus.zip      shakespeare/
biocreative_ppi.zip      mac_morpho/            shakespeare.zip
brown/                   mac_morpho.zip         sinica_treebank/
brown.zip                machado.zip            sinica_treebank.zip
brown_tei/               masc_tagged.zip        smultron/
brown_tei.zip            movie_reviews/         smultron.zip
cess_cat/                movie_reviews.zip      state_union/
cess_cat.zip             mte_teip5/             state_union.zip
cess_esp/                mte_teip5.zip          stopwords/
cess_esp.zip             names/                 stopwords.zip
chat80/                  names.zip              subjectivity/
chat80.zip               nps_chat/              sub

In [19]:
! unzip "/usr/share/nltk_data/corpora/wordnet.zip"

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: wordnet/
  inflating: wordnet/lexnames        
  inflating: wordnet/data.verb       
  inflating: wordnet/index.adv       
  inflating: wordnet/adv.exc         
  inflating: wordnet/index.verb      
  inflating: wordnet/cntlist.rev     
  inflating: wordnet/data.adj        
  inflating: wordnet/index.adj       
  inflating: wordnet/LICENSE         
  inflating: wordnet/citation.bib    
  inflating: wordnet/noun.exc        
  inflating: wordnet/verb.exc        
  inflating: wordnet/README          
  inflating: wordnet/index.sense     
  inflating: wordnet/data.noun       
  inflating: wordnet/data.adv        
  inflating: wordnet/index.noun      
  inflating: wordnet/adj.exc         


In [20]:
%cd /kaggle/working

/kaggle/working


In [22]:
lematizer = WordNetLemmatizer()
lematizer.lemmatize("goes")

'go'

In [23]:
def tokenize(text):
    return [lematizer.lemmatize(w) for w in word_tokenize(text) if not w in stop_words]
sub_data['naration_plain']=sub_data['naration_plain'].str.lower()
sub_data.head()

,NARATION,DR_CR,naration_plain
0,Cash Withdraw,Debit,cash withdraw
1,Cash Withdrawal From A/C No.:,Credit,cash withdrawal from ac no
2,"Cash withdraw from Micro Merchant point, MM A/C",Debit,cash withdraw from micro merchant point mm ac
3,Credit Against Merchant Cash Withdraw Service ...,Credit,credit against merchant cash withdraw service ...
4,"Cash withdraw from Micro Merchant point, MM A/C",Debit,cash withdraw from micro merchant point mm ac


In [24]:
sub_data['NARATION'] = sub_data['naration_plain'].apply(tokenize)
sub_data.head()

,NARATION,DR_CR,naration_plain
0,"[cash, withdraw]",Debit,cash withdraw
1,"[cash, withdrawal, ac]",Credit,cash withdrawal from ac no
2,"[cash, withdraw, micro, merchant, point, mm, ac]",Debit,cash withdraw from micro merchant point mm ac
3,"[credit, merchant, cash, withdraw, service, ac]",Credit,credit against merchant cash withdraw service ...
4,"[cash, withdraw, micro, merchant, point, mm, ac]",Debit,cash withdraw from micro merchant point mm ac


In [25]:
tokens_trx=sub_data[['NARATION','DR_CR']].copy()
# tokens_trx.to_csv('token_nar_tran.csv',index=False)

In [26]:
tokens_trx.head()

,NARATION,DR_CR
0,"[cash, withdraw]",Debit
1,"[cash, withdrawal, ac]",Credit
2,"[cash, withdraw, micro, merchant, point, mm, ac]",Debit
3,"[credit, merchant, cash, withdraw, service, ac]",Credit
4,"[cash, withdraw, micro, merchant, point, mm, ac]",Debit


In [27]:
tlist=tokens_trx['NARATION'].tolist()
tlist[:10]

[['cash', 'withdraw'],
 ['cash', 'withdrawal', 'ac'],
 ['cash', 'withdraw', 'micro', 'merchant', 'point', 'mm', 'ac'],
 ['credit', 'merchant', 'cash', 'withdraw', 'service', 'ac'],
 ['cash', 'withdraw', 'micro', 'merchant', 'point', 'mm', 'ac'],
 ['credit', 'merchant', 'cash', 'withdraw', 'service', 'ac'],
 ['cash', 'deposit', 'ac'],
 ['cash', 'deposit'],
 ['cash', 'deposit', 'ac'],
 ['cash', 'deposit']]

In [28]:


unique_values = list(set(tlist))

counts = {value: tlist.count(value) for value in unique_values}

print(counts)


TypeError: unhashable type: 'list'

In [29]:
from gensim.models import Word2Vec

# Assuming 'tokens' is a list of tokenized documents
model = Word2Vec(tlist,min_count=1)
# X = [model[token] for token in tokens]



In [30]:
# X = [model[token] for token in tlist]
print(model)

Word2Vec<vocab=38413, vector_size=100, alpha=0.025>


In [37]:
t_words=model.wv.get_normed_vectors()
print(t_words[:2])

[[-0.04579528 -0.04279594  0.13148424 -0.15906066 -0.13874872  0.11380135
  -0.00413535  0.01194396  0.07269645  0.0736906  -0.09691998  0.07968978
  -0.11491624 -0.05576385  0.02490416 -0.01973887 -0.20626676 -0.27393833
   0.01038102  0.09363174 -0.09699154  0.07412264  0.11334708  0.02653718
   0.13078101  0.02522173  0.00106536  0.09711324 -0.12146143  0.09261862
   0.23675686  0.01333009  0.06095351  0.09028055 -0.13228357 -0.08507924
   0.03750787  0.04219766  0.02071914 -0.20364428 -0.09552959 -0.08044489
  -0.02114812  0.16472773  0.00591782  0.0094729  -0.10195822 -0.05490432
   0.0550547   0.07661172  0.25968307  0.11493555 -0.06753312 -0.11419713
  -0.03075129 -0.07600169 -0.08188459  0.07613176  0.10243466 -0.05920432
   0.00525333  0.05327747 -0.13876536  0.06153561 -0.19018576  0.06350341
  -0.04479881  0.12044188 -0.01618333 -0.07685017  0.18226816 -0.14523287
   0.02528662 -0.03377763 -0.06010434  0.15160105  0.01598492 -0.09593287
  -0.05731843  0.00482979  0.01370883 

In [40]:
print(model.wv.get_vector("withdraw", norm=True))

[-1.09169886e-01 -3.09132617e-02  2.52393365e-01 -1.10008791e-01
 -2.37798050e-01  8.69068429e-02 -1.35446563e-02 -1.38242260e-01
  4.64960895e-02  1.06204093e-01 -1.34188324e-01 -1.37743233e-02
 -7.28400201e-02 -5.85431643e-02  9.12356824e-02  4.29509170e-02
 -3.43375094e-02 -2.22457469e-01 -4.91656363e-03  2.09788419e-02
 -1.04220100e-01  3.14535201e-02  5.07224817e-03 -4.00542095e-02
  8.23199823e-02 -6.86818585e-02  6.25008121e-02  1.10633269e-01
 -1.20794550e-01  5.40021732e-02  1.27028540e-01 -2.60514207e-04
  5.70001788e-02  3.55748534e-02 -1.69694051e-02  1.04523525e-01
  5.41992579e-03  4.15127128e-02 -5.04293889e-02 -2.59091198e-01
  1.29899316e-04  4.20142114e-02  7.07060471e-02  3.98464724e-02
  5.50375916e-02 -1.64921582e-02  5.61855398e-02  6.41277730e-02
 -6.10503405e-02  1.64381132e-01  1.36138588e-01  6.78583756e-02
 -2.90276259e-02  1.68485194e-02  2.14178693e-02 -1.10640779e-01
 -9.78497416e-02  9.23117846e-02 -1.24776058e-01  1.18384324e-01
  6.42412901e-02  1.04811

In [48]:
print(model.wv.most_similar('ac'))

[('cash', 0.528668999671936), ('withdraw', 0.4675011932849884), ('merchant', 0.4351734519004822), ('purchase', 0.3898284137248993), ('micro', 0.3559021055698395), ('withdrawcow', 0.3278512954711914), ('himeltangail', 0.3220791816711426), ('depositdmd', 0.3219985365867615), ('mal', 0.3162429630756378), ('eram', 0.3038964569568634)]


In [ ]:
def vectorize(text):
    return model.wv.get_vector(text, norm=True)

In [51]:
tokens_trx['vectorized']=tokens_trx['NARATION'].apply()

'cash'